<h2>Load the data</h2>

In [17]:
import os
import re
import sys
import glob
import numpy as np
import pandas as pd
from nltk import tokenize

In [18]:
txt_files = []
for name in glob.glob('ElecDeb60To16/*.txt'):
    name = name.replace('\\','/')
    txt_files.append(name)

ann_files = []
for name in glob.glob('ElecDeb60To16/*.ann'):
    name = name.replace('\\','/')
    ann_files.append(name)

In [19]:
sent_all = []
ind_all = []

for file in txt_files:
    sent_list = []
    ind_list = []
    prev_ind = 0
    
    txt_file = open(file)
    text = txt_file.read()
    txt_file.close()
    
    
    with open(file) as infile:
        for line in infile:
            line = re.sub(r'(\([A-Z]+\)_[A-Z]+(\_[A-Z]+)?: )','',line)
            line = re.sub(r'([A-Z]+\_[A-Z]+\_[A-Z]+: )','',line)
            line = re.sub(r'([A-Z]+\_[A-Z]+: )','',line)
            line = re.sub(r'(\_[A-Z]+: )','',line)
            line = re.sub(r'([A-Z]+: )','',line)
            line = re.sub(r'\[[a-zA-Z0-9;,\s\"\'\.\S]*\]*','',line)
                  
            for sent in tokenize.sent_tokenize(line):
                if not sent_list:
                    ind = text.find(sent,0,len(text)) 
                else:
                    ind = text.find(sent,prev_ind,len(text))
                prev_ind = ind
                sent_list.append([sent,ind])
                ind_list.append(ind)
                
    ind_all.append(np.array(ind_list))
    sent_all.append(sent_list)

#Up to this point we get a sent_all containing every debate (42 total) tokenized into sentences.
#sent_all = [sent_list, sent_list, sent_list,...]
#sent_list = [[sent1, index],[sent2, index],[sent3,index],...]
#ind_all = [ind_list, ind_list, ind_list,...]
#ind_list = [ind, ind, ind,...]

In [20]:
print(ind_all)

[array([    9,    75,   250,   509,   596,   613,   633,   684,   747,
         783,   810,   869,   959,  1115,  1158,  1249,  1306,  1371,
        1426,  1505,  1536,  1690,  1818,  1868,  2030,  2131,  2226,
        2266,  2300,  2453,  2476,  2494,  2513,  2604,  2686,  2776,
        2798,  2829,  2978,  3012,  3037,  3147,  3229,  3433,  3501,
        3580,  3671,  3831,  3902,  3932,  3968,  4021,  4169,  4263,
        4357,  4416,  4570,  4713,  4817,  4911,  4990,  5028,  5135,
        5340,  5579,  5686,  5779,  5818,  5931,  6032,  6089,  6322,
        6476,  6634,  6953,  7140,  7243,  7318,  7455,  7620,  7731,
        7919,  8132,  8141,  8287,  8327,  8439,  8503,  8545,  8796,
        8859,  8920,  9011,  9054,  9203,  9277,  9297,  9372,  9388,
        9433,  9474,  9536,  9624,  9674,  9714,  9781,  9813,  9863,
        9917, 10023, 10121, 10213, 10326, 10418, 10578, 10757, 10815,
       10886, 10905, 10972, 11064, 11118, 11257, 11361, 11495, 11566,
       11606, 11749

In [21]:
ann_all = []

for file in ann_files:
    ann_list = []
    
    with open(file) as infile:
        for line in infile:
            entry = line.split("\t")
            ann_list.append(entry)
    
    ann_all.append(ann_list)

In [22]:
#following part creates labels for task 1 (whether a sentence contains an argument component)

labels_arg_all = []

for i in range(len(ind_all)):
    labels_arg_arr = np.empty([len(ind_all[i]),2])
    sent_start_ind_list = []

    #column 0 of labels_arg are the starting indices of the senteces 
    for j in range(len(ind_all[i])):
        labels_arg_arr[j,0] = ind_all[i][j]
        
    #loop through each annotation entry, get the corresponding sentence starting index 
    for j in range(len(ann_all[i])):
        phrase_start = re.search(r'[0-9]+\s',ann_all[i][j][1]).group(0)
        print(phrase_start)
        sent_start_ind_list.append(ind_all[i][ind_all[i] <= int(phrase_start)].max())
        
        phrase_start = re.search(r';[0-9]+\s',ann_all[i][j][1])
        if phrase_start != None:
            phrase_start = phrase_start.group(0)[1:]
            ind2 = ind_all[i][ind_all[i] <= int(phrase_start)].max()
            if ind2 not in sent_start_ind_list:
                sent_start_ind_list.append(ind2)
    
    #loop through starting indices for sentences, check if each sentence contains an annotation (labeled as 1)
    for j in range(len(ind_all[i])):   
        if ind_all[i][j] in sent_start_ind_list:
            labels_arg_arr[j,1] = 1
        else:
            labels_arg_arr[j,1] = 0
    
    labels_arg_all.append(labels_arg_arr.astype(int))

        
#ann_all = [ann_list, ann_list, ann_list, ...]
#ann_list = [['T1', 'Claim/Premise xxx xxx', '...'], ['T2', 'Claim/Premise xxx xxx', '...'], 
    #['T3', 'Claim/Premise xxx xxx', '...'], ...]
#labels_arg_all = [labels_arg_arr, labels_arg_arr, labesl_arg_arr, ...]
#labels_arg_arr = np array of size (len(ind_list),2), no. of rows = no. of sentences in a txt file, 
    #col 0 = starting index of sentece, col 1 = label (1 for contain claim/premise, 0 for otherwise)

977 
1025 
1131 
1079 
1208 
1271 
1420 
1455 
1711 
1751 
1800 
1831 
1992 
2160 
2205 
2415 
2579 
2641 
2691 
2762 
3397 
4116 
4148 
4177 
4272 
4441 
4547 
4581 
4638 
4693 
4822 
4895 
4953 
5070 
5137 
5204 
5173 
5234 
5319 
5370 
5408 
5478 
5643 
5749 
5668 
5927 
5836 
5971 
6075 
6190 
6338 
6689 
6707 
6933 
7042 
7054 
7079 
7157 
7246 
7294 
6140 
2363 
2390 
7341 
7466 
7517 
7546 
7567 
7618 
7673 
7736 
8452 
8489 
8545 
8870 
8908 
8954 
9067 
9098 
9179 
9366 
9511 
9562 
9592 
9634 
9700 
10130 
10167 
10134 
10210 
10281 
10395 
10450 
10596 
10728 
10902 
10949 
10985 
11026 
11718 
11922 
12030 
12098 
12780 
12831 
13014 
13040 
12421 
12499 
12534 
12716 
13463 
13445 
13213 
13075 
13673 
13698 
13592 
13834 
14186 
14271 
14500 
14616 
14734 
14791 
14820 
14881 
14945 
15033 
15155 
15179 
15253 
15503 
15699 
15658 
15809 
15899 
15932 
15969 
16044 
16137 
16193 
16354 
16475 
16678 
16845 
16942 
17014 
17193 
17259 
17420 
17453 
17602 
17693 
17774 
17

74032 
73823 
74117 
74177 
74607 
74754 
74826 
74879 
75032 
74336 
74435 
74485 
75118 
75192 
75521 
76080 
75801 
75719 
75643 
77564 
77617 
77703 
77899 
78140 
77977 
78316 
78435 
78335 
78490 
78729 
78587 
79614 
79662 
79815 
79896 
80116 
80171 
80352 
80398 
80440 
80498 
80518 
80567 
80606 
80966 
81006 
80898 
82377 
82328 
82081 
82209 
82021 
81954 
81989 
81925 
81682 
81842 
81861 
81734 
80657 
77533 
77104 
77132 
1351 
1480 
1367 
1436 
1546 
1776 
2935 
1579 
1634 
1720 
1836 
1900 
2037 
2183 
2365 
2409 
2440 
2487 
2581 
2616 
3472 
3559 
3631 
3833 
4356 
3933 
4000 
4055 
4175 
4400 
4457 
4494 
4530 
4620 
3128 
3209 
3243 
4941 
5113 
5157 
5259 
5230 
5316 
5278 
5429 
5348 
5451 
5511 
5585 
5619 
5647 
5677 
5702 
5758 
5777 
6075 
6134 
5809 
6201 
6286 
6381 
6626 
6741 
6660 
6765 
7010 
7024 
7061 
7146 
7251 
7331 
7616 
7663 
7469 
7537 
7570 
7718 
8071 
8176 
8265 
8391 
8469 
8542 
9257 
9366 
9385 
9406 
9429 
9013 
8848 
8809 
9544 
9664 
9

52477 
52511 
52592 
52660 
52709 
52876 
53216 
52999 
53047 
53081 
53132 
53242 
53335 
53775 
53797 
53845 
53377 
53413 
53471 
53512 
53551 
53571 
53632 
53713 
53597 
53741 
54953 
55423 
55478 
55501 
55529 
55590 
55618 
55635 
55655 
55705 
55780 
55752 
55839 
55860 
54958 
54885 
54929 
54825 
54256 
54295 
54358 
54417 
54451 
54498 
54558 
54607 
54664 
54764 
54783 
54995 
55038 
55114 
55148 
55194 
55281 
55371 
56434 
56559 
56614 
56645 
56693 
55924 
56003 
56062 
56143 
56264 
56343 
56737 
56837 
56931 
56967 
57061 
57106 
57190 
57255 
57347 
57371 
57426 
57467 
57490 
57508 
57547 
57586 
57632 
57724 
57777 
57682 
57812 
58248 
58260 
58278 
58290 
58544 
58611 
58660 
58694 
58758 
58836 
58898 
58952 
59015 
59100 
59179 
59219 
59282 
59322 
59377 
59441 
59487 
59540 
60054 
60110 
59670 
59603 
59709 
59920 
59961 
60036 
59756 
59792 
59831 
60142 
60223 
60424 
60593 
60515 
60475 
60263 
60313 
60368 
60407 
60703 
60813 
60917 
61083 
61162 
62286 

66927 
66943 
66982 
67027 
67099 
67730 
67808 
68097 
67850 
67957 
68272 
68323 
68369 
68492 
68521 
68649 
68695 
68766 
68782 
68894 
69122 
69250 
69418 
69523 
69651 
69686 
69797 
69941 
69995 
70405 
70381 
70140 
70169 
70222 
70435 
70497 
70548 
71922 
71955 
72021 
72114 
72172 
72209 
72529 
72583 
72646 
72736 
72759 
72267 
73136 
72840 
72925 
73027 
73044 
73222 
73335 
73375 
73397 
73485 
73622 
73679 
73751 
73829 
74504 
74565 
74613 
74186 
75241 
75342 
75437 
75528 
75615 
75759 
75916 
76077 
76104 
76143 
76230 
76284 
76544 
76596 
76343 
76393 
76415 
76745 
76899 
76986 
77097 
77152 
77198 
77331 
77395 
77613 
77279 
78154 
78191 
77749 
77837 
77982 
78014 
78048 
78108 
78786 
78838 
78873 
78893 
78935 
78968 
79169 
78989 
79558 
79964 
79999 
80203 
80230 
80413 
80292 
79191 
79213 
79272 
79452 
79522 
79689 
79800 
80527 
80868 
81230 
81329 
81425 
81502 
81560 
80793 
80704 
80769 
80832 
80922 
80942 
81002 
81127 
81158 
81835 
81804 
81844 

11477 
11526 
11674 
11750 
13941 
13836 
13765 
13538 
13598 
13645 
13700 
15084 
15186 
15246 
15351 
16046 
16063 
16083 
15613 
17053 
17116 
17369 
17717 
17780 
17867 
18253 
18385 
23376 
23594 
23792 
24579 
24518 
24624 
25014 
25042 
25793 
26048 
25934 
25965 
25982 
26001 
26136 
29633 
32042 
32081 
32113 
32195 
32547 
32426 
32683 
32736 
32824 
33019 
33250 
33069 
33116 
33298 
33342 
33389 
33550 
33584 
50340 
55292 
49792 
58448 
58501 
58538 
58576 
60239 
61737 
62244 
62453 
62535 
62835 
62745 
63164 
63315 
65322 
65437 
65103 
66328 
66955 
67428 
67894 
67796 
67832 
67857 
67624 
68162 
68245 
68438 
68282 
68492 
68611 
68712 
68824 
69437 
69412 
69336 
69869 
69954 
69984 
72513 
72549 
73103 
73160 
73394 
73431 
73302 
74224 
74353 
76132 
75588 
75738 
76028 
75285 
75337 
76976 
77587 
77709 
77770 
77817 
77407 
77535 
77043 
77645 
77909 
77951 
78180 
78221 
79737 
79996 
80938 
80977 
81103 
81185 
81535 
82029 
82088 
81782 
81729 
81659 
81995 

52479 
52574 
52874 
53151 
53203 
53074 
52921 
54010 
53862 
54290 
54209 
54364 
54691 
54650 
54518 
54412 
54831 
54971 
55012 
55043 
55142 
56171 
56049 
55954 
55901 
55855 
55802 
55481 
55307 
55434 
55244 
55174 
56431 
56916 
56831 
56634 
56489 
56602 
57645 
57710 
57628 
57530 
57401 
57450 
57301 
56937 
56972 
57114 
57185 
57214 
57260 
58340 
58397 
59099 
59075 
59053 
59035 
58764 
58974 
58887 
58859 
59139 
59205 
59273 
59416 
59435 
59512 
59548 
59596 
59639 
59683 
59716 
59850 
60168 
60366 
60275 
60314 
60397 
60462 
60636 
60749 
60810 
60872 
60903 
60965 
60992 
61119 
61043 
61697 
61485 
61404 
61436 
61277 
61306 
61192 
2523 
2631 
2717 
2779 
2892 
2997 
3064 
3153 
3213 
3360 
3386 
3494 
3576 
3599 
3623 
3700 
4375 
4441 
4484 
4290 
4313 
4178 
4198 
4242 
4132 
5453 
5607 
5709 
5844 
6007 
6112 
6149 
6253 
6458 
6514 
6561 
6629 
6656 
6865 
6904 
6934 
6975 
7766 
8345 
8400 
8440 
7785 
7923 
8479 
8526 
8640 
9550 
9579 
9265 
9351 
9155 

15802 
16418 
15864 
15906 
15952 
16035 
16110 
16136 
16160 
16205 
16253 
16313 
16006 
16757 
16814 
16923 
16975 
17017 
17053 
17096 
17110 
17153 
17218 
17275 
17329 
17414 
17442 
17471 
17748 
17648 
17724 
17842 
17893 
17954 
18003 
18082 
18123 
18167 
18216 
18266 
18328 
18425 
18500 
18653 
18700 
18803 
18862 
18902 
18937 
21039 
21070 
21106 
21151 
18990 
19118 
19231 
19403 
19322 
19272 
19481 
19531 
19761 
19832 
19906 
19961 
20018 
20113 
20127 
20309 
20380 
20491 
20532 
20574 
20604 
20693 
20750 
20797 
20967 
20832 
20906 
21886 
21284 
21444 
21745 
21820 
21934 
22015 
22154 
22287 
22310 
22433 
22458 
22495 
22524 
22553 
22661 
22708 
22740 
22782 
22904 
23065 
23111 
23143 
23324 
23533 
23455 
23490 
23604 
23647 
23703 
23723 
23764 
23813 
23943 
23994 
25501 
25378 
24061 
24154 
24173 
24224 
24278 
24258 
24351 
24387 
24431 
24496 
24547 
24697 
24782 
24746 
24855 
24965 
25047 
25116 
25219 
25291 
25338 
26253 
26781 
26584 
26628 
26402 

24051 
23946 
23931 
23923 
24146 
24188 
24233 
24393 
24295 
24436 
24656 
24968 
25017 
25097 
25217 
25524 
25548 
25684 
25779 
25897 
25925 
26085 
26289 
26659 
26703 
26971 
27310 
27314 
27324 
27811 
27842 
27919 
27865 
27893 
28110 
28175 
28246 
28211 
28333 
28307 
28564 
28672 
28792 
28876 
28930 
29001 
29064 
29118 
29148 
29341 
29500 
29552 
29582 
29662 
30019 
29920 
29903 
29872 
29805 
29755 
29712 
31069 
30951 
30905 
30856 
30646 
30681 
30700 
30725 
30385 
30363 
30328 
30142 
30254 
31282 
31359 
31493 
31497 
31437 
31509 
31722 
31536 
31582 
31630 
32250 
32336 
32385 
32270 
32544 
32628 
32764 
32717 
32678 
32863 
32847 
32884 
32937 
33475 
33862 
33948 
33691 
33820 
34124 
34164 
35093 
35028 
34995 
35520 
35287 
35373 
35176 
35568 
35572 
35607 
36592 
36479 
36428 
36122 
36178 
36237 
36268 
36721 
36732 
36773 
36799 
37043 
36915 
37239 
37574 
37542 
37497 
37412 
37324 
38414 
38649 
38891 
38992 
39032 
39361 
39582 
40275 
40147 
40661 

ValueError: zero-size array to reduction operation maximum which has no identity

In [23]:
labels_cp_all = []

for i in range(len(labels_arg_all)):
    labels_cp_list = []
    
    for j in range(labels_arg_all[i].shape[0]):
        if labels_arg_all[i][j,1] != 0:
            labels_cp_list.append(labels_arg_all[i][j,0])
    
    labels_cp_arr = np.zeros((len(labels_cp_list),3))
    labels_cp_out = np.zeros((len(labels_cp_list),2))
    
    for k in range(len(labels_cp_list)):
        labels_cp_arr[k,0] = labels_cp_list[k]

        
    for j in range(len(ann_all[i])):
        #check for claim with only 1 continuous part
        phrase_start = re.search(r'c|Claim\s[0-9]+\s[0-9]+',ann_all[i][j][1])
        if phrase_start != None:
            phrase_start = phrase_start.group(0)
            labels_cp_arr[np.where(labels_cp_arr[:,0] == ind_all[i][ind_all[i] <= int(phrase_start.split()[1])].max())[0],1] +=\
            int(phrase_start.split()[2]) - int(phrase_start.split()[1])
        
        #check for claim formed from 2 separate parts
        phrase_start = re.search(r'c|Claim[0-9\s]+;[0-9]+\s[0-9]+',ann_all[i][j][1])
        if phrase_start != None:
            phrase_start = phrase_start.group(0)
            labels_cp_arr[np.where(labels_cp_arr[:,0] == ind_all[i][ind_all[i] <= int(phrase_start.split(';')[1].split()[0])].max())[0],1] +=\
            int(phrase_start.split(';')[1].split()[1]) - int(phrase_start.split(';')[1].split()[0])
        
        #check for premise with only 1 continuous part
        phrase_start = re.search(r'p|Premise\s[0-9]+\s[0-9]+',ann_all[i][j][1])
        if phrase_start != None:
            phrase_start = phrase_start.group(0)
            labels_cp_arr[np.where(labels_cp_arr[:,0] == ind_all[i][ind_all[i] <= int(phrase_start.split()[1])].max())[0],2] +=\
            int(phrase_start.split()[2]) - int(phrase_start.split()[1])
        
        #check for premise formed from 2 separate parts
        phrase_start = re.search(r'p|Premise[0-9\s]+;[0-9]+\s[0-9]+',ann_all[i][j][1])
        if phrase_start != None:
            phrase_start = phrase_start.group(0)
            labels_cp_arr[np.where(labels_cp_arr[:,0] == ind_all[i][ind_all[i] <= int(phrase_start.split(';')[1].split()[0])].max())[0],2] +=\
            int(phrase_start.split(';')[1].split()[1]) - int(phrase_start.split(';')[1].split()[0])
    
    #compare length of claim and premise components in a sentence, and label the sentence according to the longer component
    labels_cp_out[:,0] = labels_cp_arr[:,0]
    for m in range(labels_cp_arr.shape[0]):
        if labels_cp_arr[m,1] >= labels_cp_arr[m,2]:
            labels_cp_out[m,1] = 1
            
    labels_cp_all.append(labels_cp_out)
    
#labels_cp_all = [labels_cp_out, labels_cp_out, labels_cp_out, ...]
#labels_cp_out = np array of size ((len(labels_cp_list),3)), 
    #no. of rows = no. sentences that contains at least 1 claim or 1 premise,
    #col 0 = starting index of such a sentence, col 1 = label (1 for claim, 0 for premise)

In [24]:
np.set_printoptions(threshold=sys.maxsize)
print(labels_arg_all[0:50])

[array([[    9,     0],
       [   75,     0],
       [  250,     0],
       [  509,     0],
       [  596,     0],
       [  613,     0],
       [  633,     0],
       [  684,     0],
       [  747,     0],
       [  783,     0],
       [  810,     0],
       [  869,     0],
       [  959,     1],
       [ 1115,     1],
       [ 1158,     1],
       [ 1249,     1],
       [ 1306,     1],
       [ 1371,     1],
       [ 1426,     1],
       [ 1505,     0],
       [ 1536,     1],
       [ 1690,     1],
       [ 1818,     1],
       [ 1868,     1],
       [ 2030,     0],
       [ 2131,     1],
       [ 2226,     0],
       [ 2266,     0],
       [ 2300,     1],
       [ 2453,     1],
       [ 2476,     0],
       [ 2494,     0],
       [ 2513,     1],
       [ 2604,     1],
       [ 2686,     1],
       [ 2776,     0],
       [ 2798,     0],
       [ 2829,     0],
       [ 2978,     0],
       [ 3012,     0],
       [ 3037,     1],
       [ 3147,     0],
       [ 3229,     1],
       [ 3